In [ ]:
import hipopy.hipopy as hp
import numpy.ma as ma
from tqdm import tqdm
import torch
import torch_geometric as tg
from torch_geometric.data import Data, InMemoryDataset, download_url

In [ ]:
# Set params for reading data
files = "/work/clas12/jnp/cvt/out_gen_cvt_rgbbg50na.hipo"
banks = [
    "BST::HitsPos"
]
keys = [
    'BST::HitsPos_ID',
    'BST::HitsPos_sector',
    'BST::HitsPos_layer',
    'BST::HitsPos_strip',
    'BST::HitsPos_r1',
    'BST::HitsPos_theta1',
    'BST::HitsPos_phi1',
    'BST::HitsPos_r2',
    'BST::HitsPos_theta2',
    'BST::HitsPos_phi2',
    'BST::HitsPos_tstatus',
    'BST::HitsPos_rstatus'
]
step = 1
max_events = 10**2
label_keys = keys[10:]
data_keys = keys[4:10]
datalist = []

# Loop HIPO files to create graphs
for idx, batch in tqdm(enumerate(hp.iterate(files,banks=banks,step=step,experimental=True))): #NOTE: RETURNS ARRAYS WITH SHAPE (NEVENTS=1,NROWS)
    
    # Check labels tstatus and rstatus
    rec_mask = torch.eq(batch[label_keys[1]][0],1) #NOTE: ONLY GET TRACKS THAT ARE RECONSTRUCTED
    y = torch.tensor(ma.array(batch[label_keys[0]][0])[rec_mask], dtype=torch.long) #NOTE: IF YOU WANT TO DO NODE CLASSIFICATION
    y = torch.tensor([1 if y.sum().item()==y.shape[0] else 0],dtype=torch.long) #NOTE: IF YOU JUST WANT ALL THE TRACKS TO BE TRUE

    # Get data arrays
    x = torch.moveaxis(torch.tensor([ma.array(batch[key][0])[rec_mask] for key in data_keys], dtype=torch.float),[0,1],[1,0])
    nnodes = x.shape[0] #NOTE: AFTER TORCH.MOVEAXIS ABOVE x.shape should = (NNODES,NFEATURES)
    if nnodes<=0: continue
    
    # Create graph
    edge_index = torch.tensor([[i for i in range(nnodes)],[i+1 if i<nnodes-1 else 0 for i in range(nnodes)]],dtype=torch.long)
    data = Data(x=x, edge_index=edge_index, y=y)
    
    # Sanity check
    if torch.max(edge_index)>=nnodes or torch.min(edge_index)<0:
        print("DEBUGGING: ERROR: torch.max(edge_index)>=nnodes")
        print("DEBUGGING: edge_index = ",edge_index)
        print("DEBUGGING: nnodes = ",nnodes)
        print("DEGBUGGING: idx = ",idx)
    if edge_index.shape[1]!=nnodes:
        print("DEBUGGING: ERROR: edge_index.shape[1]!=nnodes")
        print("DEBUGGING: edge_index = ",edge_index)
        print("DEBUGGING: nnodes = ",nnodes)
        print("DEGBUGGING: idx = ",idx)
    
    # Add graph to list
    datalist.append(data)


In [ ]:
# Define dataset class
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None, data_list=None):
        self.data_list = data_list
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        data_list = self.data_list

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# Create dataset
root = "pyg_datasets" #NOTE: DATA WILL BE SAVED IN <root>/processed/data.pt
mydataset = MyOwnDataset(root, transform=None, pre_transform=None, pre_filter=None, data_list=data_list)
print(mydataset[0])
print(len(mydataset))#NOTE: YOU SHOULD SEE the Processing...\nDone! message from dataset.process() being called.

In [ ]:
!du -sh $root
!ls -lrth $root/processed/data.pt